In [4]:
import requests

url = "http://127.0.0.1:5000/remove_duplicates?day_range=1"

data = {
    "Message": ["Привіт", "Як справи?", "Привіт"],
    "MessageDate": ["2029-01-28 19:28:10.123", "2029-01-29 10:00:00.000", "2029-01-28 19:28:10.123"],
    "TelegramPostInfoID": [1, 2, 3]
}

response = requests.post(url, json=data)
print(response.json())



{'unique_ids': [1, 2]}


In [33]:
import requests
import sqlite3
url = "http://127.0.0.1:5001/api/fetch_posts"  
payload = {
    "channel": "news_channel",
    "start_date": "2024-02-01",
    "end_date": "2024-02-05",
    "model": "Bert"  
}

headers = {"Content-Type": "application/json"}

response = requests.post(url, json=payload, headers=headers)

print(response.status_code)
print(response.json())  


200
{'message': 'Повідомлення успішно отримані та збережені'}


In [4]:
DB_PATH = "/Users/ulanagusar/Desktop/ML_week/ML_week_nlp_solving/backend/database/database.db"
print(DB_PATH)
import sqlite3
def get_db_connection():

    conn = sqlite3.connect(DB_PATH)  
    conn.row_factory = sqlite3.Row 
    return conn


def fetch_table_contents(table_name):

    try:
        conn = get_db_connection()
        cursor = conn.cursor()


        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        conn.close()

        if rows:

            for row in rows:
                print(dict(row)) 
        else:
            print(f" {table_name} empty.")
    
    except Exception as e:
        print(f"{e}")
def show_all_tables():
    fetch_table_contents("TelegramPostInfo")



/Users/ulanagusar/Desktop/ML_week/ML_week_nlp_solving/backend/database/database.db


In [5]:
show_all_tables()


📌 Вміст таблиці TelegramPostInfo:
{'MessageID': 5, 'Message': 'Росія планує наступ завтра', 'Channel': 'c1', 'MessageDate': '2029-01-28 19:28:10.123', 'Name': 'name', 'Location': 'location', 'Weapons': 'weapons', 'Observation': 'o', 'Discussion': 'd', 'Conclusion': 's', 'Recommendation': 'r'}
{'MessageID': 6, 'Message': 'Бои под Авдєєвкой продолжаются , скоро откроют новую продвижную дорогу , по которой пройдет наш пихотний батальйон', 'Channel': 'c2', 'MessageDate': '2029-01-29 10:00:00.000', 'Name': 'name', 'Location': 'location', 'Weapons': 'weapons', 'Observation': 'o', 'Discussion': 'd', 'Conclusion': 's', 'Recommendation': 'r'}
{'MessageID': 7, 'Message': 'Получили новое вооружение - гранати', 'Channel': 'c3', 'MessageDate': '2029-01-28 19:28:10.123', 'Name': 'name', 'Location': 'location', 'Weapons': 'weapons', 'Observation': 'o', 'Discussion': 'd', 'Conclusion': 's', 'Recommendation': 'r'}


In [3]:
import requests
import sqlite3

API_URL = "http://127.0.0.1:5001/api/posts"  

def test_get_posts():

    try:
        response = requests.get(API_URL)  
        if response.status_code == 200:
            posts = response.json()

            for post in posts:
                print(post)
        else:
            print(f" {response.status_code}, response: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f" {e}")
test_get_posts()

✅ Отримані повідомлення:
{'Channel': 'c1', 'Message': 'Росія планує наступ завтра', 'MessageDate': '2029-01-28 19:28:10.123', 'TelegramPostInfoID': 5}
{'Channel': 'c2', 'Message': 'Бои под Авдєєвкой продолжаются , скоро откроют новую продвижную дорогу , по которой пройдет наш пихотний батальйон', 'MessageDate': '2029-01-29 10:00:00.000', 'TelegramPostInfoID': 6}
{'Channel': 'c3', 'Message': 'Получили новое вооружение - гранати', 'MessageDate': '2029-01-28 19:28:10.123', 'TelegramPostInfoID': 7}
